###1.Set up enviroment

####Install packages

In [1]:
!pip install orjson==3.9.14
!pip install pdfplumber==0.9.0
!pip install unidecode
!pip install streamlit
!pip install groq
!pip install ollama
!pip install pinecone-client
!pip install langchain
!pip install langchain_community
!pip install langchain_groq
!pip install python-dotenv
!pip install pymupdf
!pip install colab-xterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 323.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successful

####Connect servers


In [47]:
# Setting up ollama server
%load_ext colabxterm
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > ollama.log 2>&1 &
!ollama pull nomic-embed-text

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm
>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
removing any unus

###2.Database

In [40]:
%%writefile database.py
# Import required tools
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from unidecode import unidecode
from pinecone import Pinecone
from pinecone import Vector
import pdfplumber
import os

# Define up required API Keys
os.environ["PINECONE_API_KEY"] = "ea466f40-b247-43cb-8d43-9d204b664aa0"

# Set up required API Keys
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Function to load pdf documents
def load_document(file):
  # load pdf file and extract text
  document = []
  with pdfplumber.open(file) as pdf:
      for page in pdf.pages:
          document.append(page.extract_text())
  # join all pages' text into a single string
  text = "\n".join(document)
  return text


# Function to chunk a document
def split_document(text):
  # define a splitter strategy
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
  # split the text document into chunks
  chunks = text_splitter.split_text(text)
  return chunks


# Function to embed text from a document pre-chunked
def embed_document(chunks):
  # define an embedding model
  embedding_model = OllamaEmbeddings(model='nomic-embed-text')
  # embed the text
  embeddings = embedding_model.embed_documents(chunks)
  return embeddings


# Function to create pinecone vectos
def create_vectors(doc_name, chunks, embeddings):
  # array to save vectors
  vectors = []
  # create a Vector instance
  for i, chunk in enumerate(chunks):
    vector_id = f"{i}-{unidecode(doc_name)}"
    vector = Vector(
        id=vector_id,
        values=embeddings[i],
        metadata={"source": unidecode(doc_name), "text": chunk}
    )
    # save vector
    vectors.append(vector)
  return vectors


# Function to upser a document vectors into a pinecone DB
def upsert_document(vectors):
  # connect to a pinecone DB
  pinecone_client = Pinecone(apikey=PINECONE_API_KEY)
  # get a pre-existed pinecone index
  pinecone_index = pinecone_client.Index(name='test')
  # define a batch size
  batch_size = 1000
  # define the total number of vectors
  total_vectors = len(vectors)
  # upsert all vectors
  for i in range(0, total_vectors, batch_size):
    batch = vectors[i:i+batch_size]
    pinecone_index.upsert(vectors=batch)

Overwriting database.py


###3.Chat

In [41]:
%%writefile chat.py
# Import required tools
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from pinecone import Pinecone
import os


# Define up required API Keys
os.environ["GROQ_API_KEY"] = "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"
os.environ["PINECONE_API_KEY"] = "ea466f40-b247-43cb-8d43-9d204b664aa0"

# Set up required API Keys
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# Function to load a LLM
def load_llm():
  # create llm instance
  llm = ChatGroq(
  groq_api_key=GROQ_API_KEY,
  model_name='llama-3.1-8b-instant',
  temperature=0.25,
  max_tokens=1024
  )
  return llm


# Function to create a vectorstore from a pinecone DB
def create_vectorstore():
  # connect to a pinecone DB
  pinecone_client = Pinecone(apikey=PINECONE_API_KEY)
  # get a pre-existed pinecone index
  pinecone_index = pinecone_client.Index(name='test')
  # define an embedding model
  embedding_model = OllamaEmbeddings(model='nomic-embed-text')
  # create a vectorstore
  vectorstore = LangchainPinecone(pinecone_index, embedding_model.embed_query, "text")
  return vectorstore


# Function to create a memory instance
def create_memory():
  # configure memory object
  memory = ConversationBufferMemory(memory_key="chat_history", output_key='answer', return_messages=True)
  return memory


# Function to create a refined prompt
def create_prompt():
  # system_prompt template
  system_template = (
      "Your name is Boto and you are my personal AI assistant, you were created to help in anything. You always respond in English.\n\n"
      "Based on the following retrieved information, the chat history, and the user's question, provide accurate and detailed responses."
  )
  # human prompt template
  human_template = (
      "retrieved information: {context}\n"
      "chat history: {chat_history}\n"
      "user query: {question}\n"
      "Detailed answer:"
  )
  # create system prompt instance
  system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
  # create human prompt instance
  human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
  # create convesation chain prompt
  qa_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
  return qa_prompt


def create_qa_chain(llm, vectorstore):
  # initialize a memory
  memory = create_memory()
  # create a prompt
  qa_prompt = create_prompt()
  # create a qa conversation chain
  qa_chain = ConversationalRetrievalChain.from_llm(
      llm=llm,
      return_source_documents=True,
      retriever=vectorstore.as_retriever(),
      memory=memory,
      verbose=False,
      chain_type='stuff',
      output_key='answer',
      combine_docs_chain_kwargs={"prompt": qa_prompt}
  )
  return qa_chain

Overwriting chat.py


###4.App

In [45]:
%%writefile app.py
# Import required tools
from database import load_document, split_document, embed_document, create_vectors, upsert_document
from chat import load_llm, create_vectorstore, create_memory, create_prompt, create_qa_chain
import streamlit as st


def streamlit_chat():
  # APP HEADER
  st.set_page_config(page_title="Boto - Your virtual AI-assistant", page_icon="🤖", layout="centered")
  st.title('Boto - Your assistant 🤖')

  # SIDEBAR
  with st.sidebar:
    # header
    st.header("Load documents")
    # upload documents item
    uploaded_files = st.file_uploader("Drag and drop your documents here (PDF)", accept_multiple_files=True, type=['pdf'])

    # UPLOAD DOCUMENTS
    if uploaded_files:
      # display process header
      if st.button("Process Documents"):
          for file in uploaded_files:
              with st.spinner(f'Processing {file.name}...'):
                  # extract text
                  text = load_document(file)
                  # split text into chunks
                  chunks = split_document(text)
                  # embed chunks
                  embeddings = embed_document(chunks)
                  # create vectors instances
                  vectors = create_vectors(file.name, chunks, embeddings)
                  # upsert documents into a predefined db
                  upsert_document(vectors)
              # display process end
              st.success(f"{file.name} Processed and successfully uploaded!")

          # re-start conversation to include new file in context
          if "qa_chain" in st.session_state:
              del st.session_state.qa_chain

  # CHAT
  if "messages" not in st.session_state:
    st.session_state.messages = []
  if "qa_chain" not in st.session_state:
    # load LLM (default: llama 3.1 8b)
    llm = load_llm()
    # create vectorstore instance from DB
    vectorstore = create_vectorstore()
    # create conversation chain
    st.session_state.qa_chain = create_qa_chain(llm, vectorstore)
  # Clear chat item
  if st.button("New conversation"):
    st.session_state.messages = []
    st.success("¡New conversation starts from now!")
  # Chat logic
  for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
  # user prompt
  if prompt := st.chat_input("Tú: "):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    # bot answer
    with st.chat_message("assistant"):
        response_container = st.empty()
        result = st.session_state.qa_chain.invoke({"question": prompt})
        response = result['answer']
        response_container.markdown(response)
        st.session_state.messages.append({"role": "assistant", "content": response})
        # show reference documents
        if 'source_documents' in result:
          with st.expander("Reference documents"):
              unique_docs = set()
              for doc in result['source_documents']:
                  source = doc.metadata['source']
                  if source not in unique_docs:
                      unique_docs.add(source)
                      st.write(source)
# run app
streamlit_chat()

Overwriting app.py


###5.Deploy

In [43]:
import urllib

# get localtunnel IP
print("Password IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password IP for localtunnel is: 34.80.191.219


**To access the application, open the link displayed after running the following cell ("your URL is: https://...") and copy and paste the local tune password on the page to gain access.**

In [48]:
# launch app
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.191.219:8501

your url is: https://spicy-candles-say.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py:92: LangChainDeprecationWarning: Importing Pinecone from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import Pinecone

with new imports of:

>> from langchain_community.vectorstores import Pinecone
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/v0.2/docs/versions/v0_2/>
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package a